# Example GDD 📼

Input: ```sentences_nlp352```

In [1]:
import yaml
import psycopg2
import requests
import pandas as pd

from utils import connect_db, get_dams

In [2]:
# Get dam names
dams = get_dams()

# Database connection
df = connect_db()
df.head()

Sentences: 147287


,docid,sentid,wordidx,words,poses,ners
0,5705014ccf58f18a4c0d6d61,1,"[1, 2, 3, 4]","[Eos, ,, Vol, .]","[NNS, ,, NNP, .]","[O, O, O, O]"
1,5705014ccf58f18a4c0d6d61,2,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[94, ,, No., 10, ,, 5, March, 2013, PAGE, 104,...","[CD, ,, NN, CD, ,, CD, NNP, CD, NN, CD, JJ, JJ...","[NUMBER, O, O, NUMBER, O, DATE, DATE, DATE, O,..."
2,5705014ccf58f18a4c0d6d61,3,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[Most, studies, of, solute, transport, through...","[JJS, NNS, IN, JJ, NN, IN, NNS, VBP, VBN, IN, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
3,5705014ccf58f18a4c0d6d61,4,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[However, ,, small, impoundments, ,, such, as,...","[RB, ,, JJ, NNS, ,, JJ, IN, DT, VBN, IN, NN, C...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,5705014ccf58f18a4c0d6d61,5,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[As, these, small, systems, mature, ,, the, im...","[IN, DT, JJ, NNS, VBP, ,, DT, NNS, VBP, IN, NN...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


## Utility Functions

In [4]:
def remap_sent(sent): return ' '.join(sent)

def n_sents(idx, df):
    ''' Returns the surrounding sentences in rel to dataframe'''
    start = idx
    end = idx
    if idx > 0:
        start = idx-1
    if idx < len(df):
        end = idx+1
    return(start, end)

## Finding Candidate Passages

In [5]:
cand_idx = []

for idx, i in enumerate(df['docid']):
    doc, sentid, wordidx, words, poses, ners = df.loc[idx]
    
    if 'Dam' in words or 'dam' in words and 'DATE' in ners:
        cand_idx.append(idx)

print('Sample: %s' %remap_sent(df['words'].iloc[cand_idx[1]]))
%time len(cand_idx)

Sample: Dam Removal Research ; Status and Prospects .
CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.25 µs


3150

## Labeling functions

From snorkel tutorial work 

```python

CandidateExtractor(Dam_Removal_Year, [ngrams, ngrams], [DateMatcher(), DictionaryMatch(d=rm)])


def LF_removal_present(c):
    rm = ['remove', 'removal', 'breach', 'destroyed', 'destroy', 'failed', 
          'removed', 'breached', 'removing', 'post-dam', 'demolition', 'demolish',
          'demolished', 'razing', 'razed', 'raze']
    for i in rm:
        if i in get_left_tokens(c, window=20) or i in get_right_tokens(c, window=20):
            return 1
    return 0

def LF_timeframe(c):
    ''' LF to ensure the dam removal is within a timeframe'''
    try: 
        c = int(c.year.get_span())
        if c > 1890 and c < 2020:
            return 1
        else: return 0
    except:
        return 0
    
    
def LF_remove_in_sentence(c):
    """A simple example of a labeling function"""
    return 1 if 'remove' in c[1].get_parent().words or 'removal' in c[1].get_parent().words else 0
```

In [6]:
def removal_present(series):
    rm = ['remove', 'removal', 'breach', 'destroyed', 'destroy', 'failed', 
          'removed', 'breached', 'removing', 'post-dam', 'demolition', 'demolish',
          'demolished', 'razing', 'razed', 'raze']
    
    doc, sentid, wordidx, words, poses, ners = series
    
    for i in rm:
        if i in words:
            return 1
    return 0

removal_present(df.iloc[6])

1

#### Sample 

In [7]:
flagged = []

for i in cand_idx:
    start, end = n_sents(i, df)
    
    for sent in [start, i, end]:
        if removal_present(df.iloc[sent]) == 1:
            flagged.append(i)

In [8]:
print('%s labels found' %len(flagged))

remap_sent(df['words'].iloc[flagged[0]])

1828 labels found


'-LRB- Journal of Geophysical Research‐Biogeosciences , doi :10.1029 / 2012JG002148 , 2013 -RRB- -- EB Powers et al. investigated solute transport at this site in an agricultural region of Wisconsin -LRB- left -RRB- before and -LRB- right -RRB- after dam removal .'